# Studying counts of p-rough numbers $\Phi(x,p)$

The p-rough numbers are the numbers all of whose prime factors are greater than the prime $p$.  The unit $1$ is included as a $p$-rough
number for all primes $p$.

The cycle of gaps $\mathcal{G}(p^\#)$ are exactly the gaps between the $p$-rough numbers.

The count of $p$-rough numbers up through $x$ is denoted $\Phi(x,p)$.

From the cycle $\mathcal{G}(p^\#)$ we know that
- $\Phi(x + p^\#, p) = \Phi(x,p) + \phi(p^\#)$
- $\Phi(k \cdot p^\#, p) = k\cdot \phi(p^\#)$
- $\Phi((k+\frac{1}{2}) \cdot p^\#, p) = (k+\frac{1}{2}) \cdot \phi(p^\#)$

Thus the graph of $\Phi(x,p)$ has a line of symmetry
$$ \tilde{\Phi}(x) = \frac{\phi(p^\#)}{p^\#} x = \frac{1}{\mu} x $$
where $\mu = \frac{p^\#}{\phi(p^\#)}$ is the mean size of the gaps in
$\mathcal{G}(p^\#)$.

There is a simplicity in working with 
$$ \Delta \Phi(x,p) = \Phi(x,p) - \frac{1}{\mu} x$$
This measures the deviation of $\Phi(x,p)$ around its line of symmetry.
The symmetries above become
- $\Delta \Phi(x + p^\#, p) = \Delta \Phi(x,p)$
- $\Delta \Phi(k \cdot p^\#, p) = 0$
- $\Delta \Phi((k+\frac{1}{2}) \cdot p^\#, p) = 0$
- $\Delta \Phi(p^\# -x , p) = -\Delta \Phi(x , p)$

The deviations of the $p$-rough numbers from their line of symmetry
$$\tilde{\Phi} = \frac{1}{\mu}x$$ 
are periodic and bounded.  Thus the behavior of $\Phi(x,p)$ is completely described by the behavior of $\Delta \Phi(x,p)$
over the first cycle $\mathcal{G}(p^\#)$, or using the rotational symmetry around $x=1+\frac{p^\#}{2}$ over the first
half of this cycle.


In [1]:
%reset -f

# This version uses pandas & matplotlib to provide interactive graphics
import pandas as pd
import numpy as np
import array
import itertools
import matplotlib
%matplotlib inline
# matplotlib.use("nbagg")

import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
plt.ion

import gc
import psutil
import sys

The array DelPhi contains the lower points on the vertical segments for $\Delta \Phi(x,p)$.  The upper points on these segments are (DelPhi+1)

In [2]:
# block to check the available system memory
gc.collect()
memory = psutil.virtual_memory()
available_memory = memory.available
del memory
print(f"Available memory: {available_memory / (1024 ** 2):.2f} MB")

Available memory: 6079.47 MB


In [3]:
# Recursive construction of cycles, from 01_Cycles.ipynb
def R123(Gin):
    """
    Performs the 3-step recursion on a cycle G(p#). 
    Returns the next cycle of gaps.
    """
    gc.collect()
    newp = int(Gin[0]+1)
    lenG = int((newp -1) * len(Gin))
    lenGin = len(Gin)
    print("length ", lenG, "for cycle G(",newp ,"#)")
    try:
        Gout = array.array('H', [0]*lenG)
    except MemoryError:
        print("Memory allocation fail in R123 for p=", newp)
        Gout = array.array('H', [0]*4)
        return Gout
    """
    We will use Gin and a counter to track the concatentations in R2
    """
    icopy = 1  # index for the concatenated copy of Gin
    ifusion = 0  # index for the running sums newp*Gin
    Gout[0] = Gin[0]
    iin = 0  # index for copying from Gin
    iout = 0  # index for copying into Gout
    while (iout < (lenG-1)):  # This loop starts at a fusion
        targetsum = newp * Gin[ifusion]
        iin += 1
        if (iin >= lenGin):  # move into the next copy of Gin
                iin = 0
                icopy += 1
        runningsum = Gin[iin]
        Gout[iout] += Gin[iin]
        while (runningsum < targetsum and iout < (lenG-1) ):
            iin += 1
            iout += 1
            if (iin >= lenGin):  # move into the next copy of Gin
                iin = 0
                icopy += 1
            runningsum += Gin[iin]  # add to the running sum
            Gout[iout] = Gin[iin]   # copy into the new cycle
        # this point marks the next fusion
        if (runningsum > targetsum):
            print("sum error at fusion", ifusion, runningsum, "vs", targetsum)
        if (ifusion < (lenGin -1)):
            ifusion += 1
    return(Gout)

In [4]:
# We try to read G29 from file and otherwise we construct it, borrowing code from 01_Cycles.ipynb
try:
    G29 = np.load('G29uint.npy')
except FileNotFoundError:
    G19 = np.load('G19uint.npy')
    G23 = R123(G19)
    G29 = R123(G23)
    np.save('G29uint.npy',G29)


In [5]:
# average gap size for G(29#)
mu_gap = (29/28)*(23/22)*(19/18)*(17/16)*(13/12)*(11/10)*(210/48)
mu_recip = -1/mu_gap
print("mu",mu_gap, "neg reciprocal (slope)", mu_recip)

mu 6.33122875072338 neg reciprocal (slope) -0.15794722310195222


In [6]:
lenG = len(G29)
# len(G29) should be 1021870080

In [7]:
# We try to read DelPhi from file and otherwise we construct it
# The array DelPhi contains the lower points on the vertical segments.
# The upper points would be DelPhi+1, and the midpoints would be DelPhi+0.5
try:
    DelPhi = np.load('DelPhi29.npy')
except FileNotFoundError:
    DiffDelPhi = np.zeros(lenG, dtype=float)
    i=0
    while (i < lenG):
        DiffDelPhi[i] = 1 + mu_recip * G29[i]
        if ((i % 50000000) == 0):
            print(i, G29[i], DiffDelPhi[i], end='\r')
        i += 1
    DiffDelPhi = np.insert(DiffDelPhi, 0, mu_recip)
    DelPhi = np.cumsum(DiffDelPhi)
    np.save("DelPhi29.npy", DelPhi)  # save DelPhi for future use


In [8]:
print(f"G29 is {sys.getsizeof(G29)/(1024**2) :.3f} MB, length {len(G29)}")

G29 is 1949.063 MB, length 1021870080


In [9]:
print(f"DelPhi is {sys.getsizeof(DelPhi)/(1024**2) :.3f} MB, length {len(DelPhi)}")

DelPhi is 7796.250 MB, length 1021870081


In [10]:
DelPhi[0:20]

array([-0.15794722, -3.89636392, -3.84404725, -3.47583615, -2.79173059,
       -2.42351949, -2.37120282, -2.31888616, -1.63478061, -1.58246395,
       -1.21425284, -0.53014729, -0.47783063, -0.10961952, -0.05730286,
       -0.32088064,  0.04733047,  0.73143602,  1.09964713,  1.78375268])

In [11]:
# checking max value of 19.912...
np.max(DelPhi)

np.float64(19.91277999178553)

In [12]:
# at index 167215228 ...
np.where(DelPhi > 19.9)

(array([167215228]),)

In [13]:
# view of the middle of the cycle G(29#)
# gaps at mid-cycle
G29[510934950:510935130]

array([ 6,  4,  2, 10,  2, 10,  2,  4,  6,  6,  8,  6,  4,  6,  6,  8,  4,
        2,  6, 10,  8,  4,  2, 10,  8,  6,  4,  6,  2,  4,  6, 14,  4,  2,
       10,  2, 10,  2,  4,  2, 10,  2, 12,  4,  2,  4,  8,  6,  4,  6,  6,
        6,  2,  6,  4,  8, 10,  8,  4,  2,  6,  4,  8,  6, 10,  6,  6,  2,
        6, 10,  2,  4,  8,  6,  4,  2,  4, 12, 12,  8,  4,  8, 10,  2, 30,
       16,  8,  4,  2,  4,  2,  4,  8, 16, 30,  2, 10,  8,  4,  8, 12, 12,
        4,  2,  4,  6,  8,  4,  2, 10,  6,  2,  6,  6, 10,  6,  8,  4,  6,
        2,  4,  8, 10,  8,  4,  6,  2,  6,  6,  6,  4,  6,  8,  4,  2,  4,
       12,  2, 10,  2,  4,  2, 10,  2, 10,  2,  4, 14,  6,  4,  2,  6,  4,
        6,  8, 10,  2,  4,  8, 10,  6,  2,  4,  8,  6,  6,  4,  6,  8,  6,
        6,  4,  2, 10,  2, 10,  2,  4,  6,  6], dtype=uint16)

In [14]:
# Delta-Phi values on the left side of mid-cycle
DelPhi[510935025:510935055]

array([ 4.58545571,  5.26956127,  5.63777237,  4.7424057 ,  3.84703902,
        3.58346124,  3.95167234,  3.68809456,  3.10862233,  3.79272788,
        0.05431119, -1.47284438, -1.73642217, -1.36821106, -0.68410551,
       -0.3158944 ,  0.36821116,  0.73642226,  0.47284448, -1.05431109,
       -4.79272778, -4.10862223, -4.68809446, -4.95167225, -4.58346114,
       -4.84703892, -5.7424056 , -6.63777228, -6.26956117, -5.58545562])

In [15]:
def interleave_np(arr1, arr2):
    stacked_arr = np.stack((arr1, arr2),axis=1)
    return stacked_arr.flatten().tolist()

In [16]:
G29[167215100:167215230]

array([ 6,  8,  6,  6,  4, 14, 10,  2,  4,  6,  6,  2,  6,  4,  2, 10,  6,
       14,  6,  4,  6, 20, 12,  6,  4,  6,  2,  4,  8, 12,  4,  6,  8,  6,
        4,  2,  4, 12,  2, 10, 12,  6,  2,  6,  4,  6,  6,  6,  8,  4,  2,
       16,  8,  6,  4,  2,  4, 18,  6,  2,  4,  6,  2,  6, 12, 10,  2,  6,
        4, 18, 12,  2,  4,  2, 10,  2,  6,  6,  4,  6,  6,  2, 10,  2,  6,
        4, 14,  6,  4,  2,  4,  8,  6,  4,  6,  6,  6,  2, 12,  4,  2,  4,
        6,  2,  6, 22,  2, 16,  2,  4,  8,  6,  4,  6,  6,  6,  2,  6,  4,
        2,  6, 10,  8,  4,  2,  4,  2,  4, 14,  4], dtype=uint16)

In [17]:
np.mean(G29[167215100:167215230])

np.float64(6.184615384615385)

In [18]:
# function for rolling averages
def rolling_avg(arr, wlen):
    lenAvg = len(arr)-wlen+1
    Avg_array = np.zeros(lenAvg, dtype=float)
    Avg_array[0] = sum(arr[0:wlen]) / wlen
    i=1
    while (i < lenAvg):
        Avg_array[i] = Avg_array[i-1] - arr[i-1]/wlen + arr[i+wlen-1]/wlen
        i += 1
    return Avg_array

In [19]:
peaks = np.where(DelPhi > 17)
try:
    del(featurepts)
except NameError:
    i=0

midlenG = int(lenG/2)
i=0
j=0
featurepts = {'midcycle' : midlenG}
featurepts['1/29 cycle'] = int(midlenG/29)
featurepts['4/28 cycle'] = int(midlenG/7)
featurepts['8/29 cycle'] = int((8/29)* midlenG)
featurepts['12/29 cycle'] = int((12/29)*midlenG)
featurepts['max peak'] = 167215228

while(i < len(peaks[0])):
    val = peaks[0][i]
    lowval = val
    highval = val
    while ((i < len(peaks[0])) and (peaks[0][i] < (val+5000))):
        highval = peaks[0][i]
        i += 1
    featurepts[f"peaks {j}"] = int((lowval+highval)/2)
    j += 1
   

In [20]:
# create an interactive version of the plot

minwidth = 100
midlenG = int(lenG/2)
old_altmid = midlenG

def draw_DelPhi(xselmid, xaltmid, xwidth, xavgwindow):
    # Interleaving data to get the stepped graph, rendering the vertical segments
    global old_altmid
    if (xaltmid != old_altmid):
        xmid = xaltmid
        old_altmid = xaltmid
        xmidSelect.value = xaltmid
    else:
        xmid = xselmid
    pts0 = xmid-xwidth
    pts1 = xmid+xwidth
    if (pts0 < 0):
        ptshift = -pts0
        pts0 = 0
        pts1 = pts1+ptshift
    elif (pts1 > lenG):
        ptshift = pts1-lenG
        pts0 = pts0-ptshift
        pts1 = lenG
    xmid = int((pts0+pts1)/2)
    delx = np.cumsum(G29[pts0:pts1], dtype=float)
#    print(f"{pts0} x:{delx[0]} mid {xmid} end {pts1} x: {delx[-1]}")
    delphi = DelPhi[(pts0+1):(pts1+1)]
    delxB = interleave_np(delx,delx)
    delphiB = interleave_np(delphi,(delphi +1))
    data_sample = {'x': delxB, 'DelPhi': delphiB}
    df = pd.DataFrame(data_sample)

    # calculate rolling average, to smooth the graph
    halfwindow = int(xavgwindow/2)
    avgp0 = pts0 - halfwindow
    avgp1 = pts1 + halfwindow
    if (avgp0 < 0):
        avgp0 = 0
        avgp1 += halfwindow
    elif (avgp1 > lenG):
        avgp1 = lenG
        avgp0 -= halfwindow

    delphiavg = np.array(DelPhi[avgp0:avgp1])
    # DelPhi contains the lower pts on the vertical segments, we average the midpoints
    delphiC = rolling_avg((delphiavg + 0.5), xavgwindow)
#    print(f"Avg wid {xavgwindow} sample {delphiC[0:5]} vs {delphiavg[0:5]}")
    delxC = np.cumsum(G29[(avgp0+halfwindow-1):(avgp1-halfwindow)])  
    avgdata_sample = {'x': delxC, 'AvgDelPhi': delphiC}

    avgdf = pd.DataFrame(avgdata_sample)

    # plotting the two curves
    plt.clf()
    fig, ax = plt.subplots()
    fig.set_size_inches(12,9)
    ax.set_title(f"$\Delta \Phi(29^\#)$ and its rolling average around i={(pts0+pts1)/2:.0f}")
    ax.grid(axis='y', color='#080408', lw=0.125 )

    if (xwidth < 600):
        linecolor = '#1E90FF'
        linewt = 0.25
    elif (xwidth < 6000):
        linecolor = '#87CEEB'
        linewt = 0.125
    else:
        linecolor = '#87CEFA'
        linewt = 0.125
        
    ax.plot(delxB, delphiB, color=linecolor, lw=linewt, label='DelPhi')
    ax.plot(delxC, delphiC, color='#460099', lw=0.375, label='AvgDelPhi')
    ax.set_ylim(-20,20)

    plt.show()

# widgets for selecting parameters for the graph Delta Phi(29#)
xmidSelect = widgets.IntSlider(min=minwidth, max=(lenG-minwidth), step=616, value=midlenG,
                  description="Midpt view", layout=widgets.Layout(width='95%'), disabled=False)
altxmidSelect = widgets.Dropdown(options=featurepts, value=featurepts['midcycle'], description="Feature pts")
xwidthSelect = widgets.SelectionSlider(options=[50,140,250,500,1200,2400,5000,7500,10000, 16000,24000,40000,80000], value=10000, description="Zoom", layout=widgets.Layout(width='80%'), disabled=False)
xavgwidSelect = widgets.SelectionSlider(options=[8,48,120,240,480, 960, 1440, 2880,5760],
                                        value=48, description="Avg Base", layout=widgets.Layout(width='40%'), disabled=False)

interact(draw_DelPhi, xselmid=xmidSelect, xaltmid=altxmidSelect, xwidth=xwidthSelect, xavgwindow=xavgwidSelect)



interactive(children=(IntSlider(value=510935040, description='Midpt view', layout=Layout(width='95%'), max=102…

<function __main__.draw_DelPhi(xselmid, xaltmid, xwidth, xavgwindow)>

In [ ]:
np.where( abs(DelPhi) < 0.00001)

In [ ]:
smallindices = np.where( abs(DelPhi) < 0.00001)

In [ ]:
DelPhi[397070]

In [ ]:
# Engelsma example
gamma0 = 1271+2310*(5+13*(8+17*(9+19*(17+21*23))))
xEngelsma= 766785216
print(f"gamma0 {gamma0} at index {xEngelsma}")

In [50]:
gamma0/mu_gap

766785223.238905

In [52]:
# avg gap in Engelsma constellation
3242/528

6.140151515151516

In [53]:
(3242/528)/mu_gap

0.9698198812434264